## Building Algorithm 9.1

In [1]:
using UncNLPTestSet, TRS, LinearAlgebra
nlp = SelectProgram("SROSENBR")
adjdim!(nlp, 200);

┌ Info: Precompiling UncNLPTestSet [ac41c6d5-581c-4fd8-9896-caf0766f302e]
└ @ Base loading.jl:1342
┌ Warning: TODO: Debug BDQRTIC
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BDQRTIC.jl:73
┌ Warning: TODO: Issue in CUTEst? See comment https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/broydn7d.jl#L50 
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BROYDN7D.jl:109
┌ Warning: TODO: Issue in CUTEst? See https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/brybnd.jl
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BRYBND.jl:110
┌ Warning: TODO: Debug CRAGGLVY
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/CRAGGLVY.jl:75
┌ Warning: TODO: Clean up DIXMAANB (and others) implementation. See comment block
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXMAANB.jl:108
┌ Warning: TODO: Implement DIXON3DQ
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXON3DQ.jl:1
┌ Warning: TOD

In [5]:
BLAS.set_num_threads(4)

function qntrHS(nlp, x₀, Hᵢ, Δₖ = 1.0, b=4, ϵ=10e-6)
	η = 0.1
	Δₘ = 1 # out of thin air 
	n = nlp.n
	xₖ = x₀
	fₖ = obj(nlp, xₖ)
	Sₖ = orth(rand(n, 2b-1))
	gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
	Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
	Hₖ = Symmetric(Hₖ)

	k = 0
	while norm(gₖ) ≥ ϵ  # confirm norm(gₖ).. this may be computed down below 
		Qₖ = [hₖ gₖ Yₖ]  			

		# trust-region subproblem
		P = Symmetric(Qₖ'*Hₖ*Qₖ)
		h = Qₖ'*gₖ
		C = Symmetric(Qₖ'*Hₖ^2*Qₖ)
		aₖ, _ = trs_small(P, h, Δₖ, C)
		aₖ = vec(aₖ) # update TRS ... it returns a column matirx

		# handle result, i.e. step must satisfy convergence criterion
		q = Qₖ*aₖ 
		pₖ = vec(Hₖ*q)
		xₜ = xₖ + pₖ # t for trial
		fₜ = obj(nlp, xₜ) # t for trial
		ρ = (fₜ - fₖ)/(0.5*aₖ'*P*aₖ + h'*aₖ) # confirm I don't need mₖ(0) in denominator

		# adjust trust region radius?
		if ρ < η
			Δₖ = 0.25*Δₖ # should I safe gaurd this to be above a lower bound
		elseif ρ > 0.75 && norm(pₖ) ≈ Δₖ
			Δₖ = min(2Δₖ, Δₘ)
		end

		# did we take the step?
		if ρ > η
			xₖ = xₖ .+ pₖ
			fₖ = fₜ
			Sₖ = orth(Yₖ - Sₖ*(Sₖ'Yₖ))
			gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
			Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
		end
		
		# extra safeguard...
		k+=1
		if k > 50000
			println("Number of iterations: $k")
			return xₖ, gₖ
		end
	end
	println("Number of iterations: $k")
	return xₖ, gₖ
end

qntrHS (generic function with 4 methods)

In [8]:
H0 = rand(nlp.n, nlp.n)
H0 = Symmetric((H0 + H0'))

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

In [6]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 ⋮    
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0
 1.2  211.6
 1.0  -88.0

In [7]:
norm(∇fxₘ)

2291.6928240927286

It seems that we didn't take any steps. Things work much better when H0 is the identity

In [8]:
H0 = zeros(nlp.n, nlp.n) + I
H0 = Symmetric(H0)

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

Number of iterations: 14529


In [9]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 1.0  8.14034e-7
 1.0  2.9691e-7
 1.0  1.07821e-6
 1.0  1.65084e-7
 1.0  6.92358e-7
 1.0  3.57621e-7
 1.0  1.05574e-6
 1.0  1.76309e-7
 1.0  1.13231e-6
 1.0  1.38085e-7
 1.0  7.32998e-7
 1.0  3.37351e-7
 1.0  8.95913e-7
 ⋮    
 1.0  6.82906e-7
 1.0  3.6234e-7
 1.0  4.76377e-7
 1.0  4.65396e-7
 1.0  1.15833e-6
 1.0  1.251e-7
 1.0  8.50033e-7
 1.0  2.78959e-7
 1.0  9.75246e-7
 1.0  2.16461e-7
 1.0  1.0483e-6
 1.0  1.80023e-7

In [10]:
norm(∇fxₘ)

8.97523081665766e-6